In [2]:
# %pip install regex requests torch numpy transformers datasets evaluate rouge_score

In [ ]:
!nvidia-smi

In [40]:
from mingpt.bpe import BPETokenizer
from mingpt.model import GPT

import torch
from torch.utils.data import Dataset
from datasets import load_dataset
import pandas as pd
from evaluate import load

# 1. Dataset

In [41]:
class QADataset(Dataset):
    def __init__(self, data, tokenizer, block_size=128):
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.data = [
            self.format_example(data_point["instruction"], data_point["demonstration"]) for data_point in data
        ]

    def format_example(self, question, answer):
        text = f"<|human|>: {question} \n <|assistant|>: {answer} <|endoftext|>"
        tokens = self.tokenizer(text)
        tokens = tokens.squeeze(0).tolist()[:self.block_size]        
        prompt = text.split("<|assistant|>:")[0] + "<|assistant|>:"
        self.assistant_index = len(self.tokenizer(prompt).squeeze(0).tolist())

        return tokens

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Returns:
            x (torch.Tensor): Input tokens (question + answer prompt).
            y (torch.Tensor): Target tokens (shifted output).
        """
        tokens = self.data[idx]
        x = torch.tensor(tokens[:-1], dtype=torch.long)  # Exclude last token for input
        y = torch.tensor(tokens[1:], dtype=torch.long)   # Exclude first token for output

        y[:self.assistant_index] = -1  # Mask loss for assistant tokens
        
        return x, y

In [ ]:
data = load_dataset("HuggingFaceH4/helpful-instructions")
pd.DataFrame(data["train"])

In [ ]:
data["train"].select(range(4))[0]

In [ ]:
tokenizer = BPETokenizer()

train_dataset = QADataset(data["train"], tokenizer=tokenizer, block_size=1024)

x, y = train_dataset[1]
print(f"Input tokens: {x}")
print(f"Output tokens: {y}")

# 2. Model Definition

In [ ]:
model_type = 'gpt2'
device = 'cuda'

model = GPT.from_pretrained(model_type)
model.to(device)

In [45]:
def generate(prompt='', num_samples=1, steps=20, do_sample=True):

    tokenizer = BPETokenizer()
    if prompt == '':
        x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
    else:
        x = tokenizer(prompt).to(device)

    x = x.expand(num_samples, -1)

    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)
    
    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        print('\n'+'-'*80)
        print(out)

In [ ]:
generate(prompt='How do I teach kids to meditate?', num_samples=2, steps=50)

# 3. Finetuning With SFT

In [ ]:
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4
train_config.batch_size = 6
train_config.max_iters = 200
train_config.num_workers = 0
trainer = Trainer(train_config, model, train_dataset)

In [ ]:
losses = []
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
        losses.append(trainer.loss.item())
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

In [ ]:
print(losses)

In [ ]:
model.eval()

In [ ]:
generate(prompt='How do I teach kids to meditate?', num_samples=2, steps=50)

In [ ]:
# save the model
torch.save(model.state_dict(), 'models/SFT_finetuned.pth')

# 4. Evaluations

In [ ]:
# load the fine-tuned model
model.load_state_dict(torch.load('models/SFT_finetuned.pth'))
model.eval()

In [ ]:
tokenizer = BPETokenizer()

def generate_evaluate(prompt='', num_samples=1, steps=20, do_sample=True):
    if prompt == '':
        x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long).to(device)
    else:
        x = tokenizer(prompt).to(device)

    x = x.expand(num_samples, -1)

    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)

    generated_responses = []
    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        generated_responses.append(out)
    
    return generated_responses

references = []
predictions = []

num_samples = 500

for i, sample in enumerate(data["train"].select(range(num_samples))):
    prompt = sample["instruction"]
    reference = [sample["demonstration"]]
    generated_text = generate_evaluate(prompt=prompt, num_samples=1, steps=50, do_sample=True)[0]

    references.append(reference)
    predictions.append(generated_text)

### 4.1 BLEU Score

In [ ]:
bleu = load("bleu")

bleu_score = bleu.compute(predictions=predictions, references=references)
print("BLEU Score:", bleu_score)

### 4.2 Rouge Score

In [ ]:
rouge = load("rouge")

scores = rouge.compute(predictions=predictions, references=references)
print("ROUGE Scores:", scores)


In [ ]:
generate_evaluate("What is AI ?")

In [ ]:
# load the fine-tuned model

path = "/Data/zakaria.abboud/SFT_finetuned.pth"
model_type = 'gpt2'
device = 'cuda'

model = GPT.from_pretrained(model_type)

# model.load_state_dict(torch.load(path))

model.to(device)

model.eval()